<a href="https://colab.research.google.com/github/hevertonvalerio/toLearn/blob/main/app_prototipo_tolearn_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protótipo 3

#2.0

In [2]:
!pip install llama_index.llms.groq -q
!pip install llama_index -q
!pip install gradio -q
!pip install llama_index.experimental -q
!pip install fpdf -q
!pip install PyPDF2 -q
!pip install sentence_transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.

In [3]:
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core import Settings
from google.colab import userdata
import gradio as gr
import os
import sqlite3
from PyPDF2 import PdfReader

from llama_index.llms.groq import Groq
from sentence_transformers import SentenceTransformer, util

In [ ]:
# API KEY do GROQ
#api_key = os.getenv("GROQ_API")
key = userdata.get('GROQ_API')


# Configuração do modelo
#llm = Groq(model="llama3-70b-8192", api_key=api_key)
Settings.llm = Groq(model='llama3-70b-8192', api_key=key)

chat_engine = SimpleChatEngine.from_defaults()
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Start

### Crie um caderno ou abra um caderno novo

In [ ]:
# Código da interface para criar um novo caderno.

### Selecione a matéria que irá estudar

#### Criando DBs

In [4]:
# DBs 2.0

# Função para criar tabelas de questões
def create_questions_table(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS questions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT NOT NULL,
            option_a TEXT NOT NULL,
            option_b TEXT NOT NULL,
            option_c TEXT NOT NULL,
            option_d TEXT NOT NULL,
            correct_option TEXT NOT NULL,
            complete_question TEXT NOT NULL,
            complete_question_answer TEXT NOT NULL
        )
    """)
    conn.commit()

# Função para criar tabela de conteúdos
def create_content_table(conn):
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS content (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            materia TEXT NOT NULL,
            conteudo_1 TEXT NOT NULL,
            conteudo_2 TEXT NOT NULL
        )
    """)
    conn.commit()

# Função para inserir dados de questões
def insert_questions(conn, questions):
    cursor = conn.cursor()
    cursor.executemany("""
        INSERT INTO questions (question, option_a, option_b, option_c, option_d, correct_option, complete_question, complete_question_answer)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, questions)
    conn.commit()

# Função para inserir dados de conteúdos
def insert_content(conn, contents):
    cursor = conn.cursor()
    cursor.executemany("""
        INSERT INTO content (materia, conteudo_1, conteudo_2)
        VALUES (?, ?, ?)
    """, contents)
    conn.commit()

# Criando os bancos de dados
def create_databases():
    # Banco de Matemática
    math_conn = sqlite3.connect("banco_matematica.db")
    create_questions_table(math_conn)
    create_content_table(math_conn)
    insert_questions(math_conn, [
        ("Quanto é 2 + 2?", "3", "4", "5", "6", "B", "A equação y = ax + b representa uma função do ____ grau, onde 'a' é o coeficiente ____ e 'b' é o coeficiente ____.", "primeiro, angular, linear"),
        ("Qual é o quadrado de 5?", "20", "10", "25", "15", "C",  "Na equação y = ax + b, o coeficiente ____ determina a inclinação da reta, enquanto o coeficiente ____ indica o ponto onde a reta intercepta o eixo ____.", "angular, linear, y")
    ])
    insert_content(math_conn, [
        ("Matemática", "Funções do 1º Grau", "Análise detalhada de funções lineares representadas pela fórmula y = ax + b, onde a é o coeficiente angular, responsável pela inclinação da reta, e b é o coeficiente linear, que indica o ponto onde a reta intercepta o eixo y. Exemplos práticos incluem cálculos de custos, receitas e trajetórias."),
        ("Matemática", "Progressões Aritméticas (PA)", "Estudo de sequências numéricas onde a diferença entre termos consecutivos é constante, abordando a fórmula geral an = a1 + (n-1)d, onde d é a razão, e o cálculo da soma dos n primeiros termos, com aplicações em finanças e organização de padrões.")
    ])
    math_conn.close()

    # Banco de História
    history_conn = sqlite3.connect("banco_historia.db")
    create_questions_table(history_conn)
    create_content_table(history_conn)
    insert_questions(history_conn, [
        ("Quem foi o primeiro presidente do Brasil?", "Getúlio Vargas", "Dom Pedro II", "Deodoro da Fonseca", "Juscelino Kubitschek", "C", "O período conhecido como Primeira República no Brasil começou com a ____ da República e foi marcado por governos de ____ e ____.", "Proclamação, militares, civis"),
        ("Em que ano ocorreu a Proclamação da República?", "1888", "1889", "1890", "1877", "B", "A Era Vargas foi marcada por políticas ____ e desenvolvimento ____ no Brasil.", "trabalhistas, industrial")
    ])
    insert_content(history_conn, [
        ("História", "Primeira República", "Período caracterizado pela Proclamação da República e o governo de presidentes militares e civis, consolidando o regime republicano no Brasil."),
        ("História", "Era Vargas", "Estudo do governo de Getúlio Vargas, abrangendo o Estado Novo, políticas trabalhistas e desenvolvimento industrial.")
    ])
    history_conn.close()

    # Banco de Biologia
    biology_conn = sqlite3.connect("banco_biologia.db")
    create_questions_table(biology_conn)
    create_content_table(biology_conn)
    insert_questions(biology_conn, [
        ("Qual é a unidade básica da vida?", "Célula", "Átomo", "Tecido", "Órgão", "A",  "As células ____ possuem núcleo definido, enquanto as células ____ não possuem núcleo definido.", "eucarióticas, procarióticas"),
        ("O que é fotossíntese?", "Processo de respiração celular", "Transformação de luz em energia química", "Divisão celular", "Ciclo de Krebs", "B", "Os ecossistemas são sistemas de interação entre seres ____ e o ____ ambiente, essenciais para o equilíbrio ecológico.", "vivos, meio")
    ])
    insert_content(biology_conn, [
        ("Biologia", "Célula eucariótica e procariótica", "Análise das diferenças entre células com núcleo definido (eucarióticas) e células sem núcleo definido (procarióticas), abrangendo funções e estruturas."),
        ("Biologia", "Ecosistemas e biodiversidade", "Estudo das interações entre seres vivos e o meio ambiente, abordando o equilíbrio ecológico e a conservação da biodiversidade.")
    ])
    biology_conn.close()

# Criar os bancos de dados
create_databases()
print("Bancos de dados criados com sucesso!")


Bancos de dados criados com sucesso!


#### Código que disponbiliza opções de matérias

## Taxonomia de Bloom




### Lembrar

### Compreender

#### Teste prática de geração de questões por IA (2.0)

In [ ]:
#Alternativa 6



# Variáveis globais
selected_subject = ""
current_question = {}
db_map = {
    "Matemática": "banco_matematica.db",
    "História": "banco_historia.db",
    "Biologia": "banco_biologia.db",
}

# Função 0 para processar o que o aluno lembra
def oq_lembro(input_lembro):
  lembro = input_lembro
  parabens = len(input_lembro)
  return lembro, parabens


# Função 1.2 Definir contéudo que o aluno deveria lembrar:
def puxar_conteudo():
    #global current_content

    if not selected_subject:
        return "Por favor, selecione uma matéria antes de gerar perguntas."

    db_path = db_map.get(selected_subject)
    if not db_path:
        return "Banco de dados não encontrado para a matéria selecionada."

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT conteudo_2 FROM content")
        row = cursor.fetchone()
        conn.close()

        if not row:
            return "Nenhuma conteúdo disponível no banco de dados."

        # Extraindo o conteúdo da tupla
        content_1 = row[0]

        return content_1

    except sqlite3.Error as e:
        return f"Erro ao acessar o banco de dados: {str(e)}"


# Função 0.1 para processar se é bom ou ruim o que o aluno disse que lembra
def avaliar_respostas(respostas_alunos, gabaritos):
    """
    Avalia respostas dissertativas com base na similaridade semântica.

    :param respostas_alunos: Lista de respostas fornecidas pelos alunos.
    :param gabaritos: Lista de gabaritos correspondentes às perguntas.
    :return: Lista de resultados contendo similaridade, classificação e justificativa.
    """
    # 1. Obter os valores do texto das caixas de texto
    respostas_alunos = respostas_alunos  # Removendo .value
    gabaritos = gabaritos  # Removendo .value

    # 2. Transformar os valores em listas (se necessário)
    respostas_alunos = [respostas_alunos]  # Transformar em uma lista com um elemento
    gabaritos = [gabaritos]  # Transformar em uma lista com um elemento

    resultados = []

    for i, (resposta, gabarito) in enumerate(zip(respostas_alunos, gabaritos)):
        # Verificar se resposta e gabarito não são None
        if resposta is not None and gabarito is not None:
            # Calcula a similaridade semântica
            embedding_resposta = model.encode(resposta, convert_to_tensor=True)
            embedding_gabarito = model.encode(gabarito, convert_to_tensor=True)
            similaridade = util.cos_sim(embedding_resposta, embedding_gabarito).item()

            # Define a classificação com base na similaridade
            if similaridade >= 0.8:
                classificacao = "Certo"
                justificativa = "A resposta está muito alinhada ao gabarito."
            elif 0.5 <= similaridade < 0.8:
                classificacao = "Meio Certo"
                justificativa = "A resposta aborda parcialmente o gabarito."
            else:
                classificacao = "Errado"
                justificativa = "A resposta não reflete o gabarito."

            # Adiciona o resultado
            resultados.append({
                "Pergunta": i + 1,
                "Resposta do Aluno": resposta,
                "Gabarito": gabarito,
                "Similaridade": round(similaridade*100, 2),
                "Classificação": classificacao,
                "Justificativa": justificativa
            })
        else:
          print("Resposta ou gabarito inválidos (None).")  # Adicionei um print para debug
          return [{
            "Pergunta": i+1,
            "Resposta do Aluno": "Resposta inválida (None)",
            "Gabarito": "Gabarito inválido (None)",
            "Similaridade": 0,
            "Classificação": "Erro",
            "Justificativa": "Resposta ou gabarito ausentes."
        }]

    return resultados





# Função 1 para definir banco de dados com base na matéria
def define_database(subject):
    global selected_subject
    selected_subject = subject
    return f"Banco de dados definido para {db_map.get(subject, 'desconhecido')}."


# Função 1.5 para criar uma pergunta de complete a frase
def complete_frase():
    #global current_content

  if not selected_subject:
      return "Por favor, selecione uma matéria antes de gerar perguntas."

  db_path = db_map.get(selected_subject)
  if not db_path:
      return "Banco de dados não encontrado para a matéria selecionada."

  try:
      conn = sqlite3.connect(db_path)
      cursor = conn.cursor()
      cursor.execute("SELECT complete_question FROM questions")
      row = cursor.fetchone()
      conn.close()

      if not row:
          return "Nenhuma conteúdo disponível no banco de dados."

      # Extraindo o conteúdo da tupla
      frase = row[0]

      return frase

  except sqlite3.Error as e:
      return f"Erro ao acessar o banco de dados: {str(e)}"



# Função 2 para gerar uma nova pergunta
def gerar_pergunta():
    global current_question

    if not selected_subject:
        return "Por favor, selecione uma matéria antes de gerar perguntas.", [""], ""

    db_path = db_map.get(selected_subject)
    if not db_path:
        return "Banco de dados não encontrado para a matéria selecionada.", [""], ""

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT question, option_a, option_b, option_c, option_d, correct_option FROM questions ORDER BY RANDOM() LIMIT 1;")
        row = cursor.fetchone()
        conn.close()

        if not row:
            return "Nenhuma pergunta disponível no banco de dados.", [""], ""

        # Guardar a pergunta atual globalmente
        current_question = {
            "question": row[0],
            "options": [row[1], row[2], row[3], row[4]],
            "correct_option": row[5],
        }
        question, option_a, option_b, option_c, option_d, correct_option = row
        pergunta_formatada = (
            f"Pergunta: {question}\n\n"
            f"A) {option_a}\n"
            f"B) {option_b}\n"
            f"C) {option_c}\n"
            f"D) {option_d}"
        )

        return pergunta_formatada, current_question['options'], "" ,

    except sqlite3.Error as e:
        return f"Erro ao acessar o banco de dados: {str(e)}", [""], ""

# Função 3 para verificar a resposta
def verificar_resposta(selected_option):
    if not current_question:
        return "Nenhuma pergunta disponível. Gere uma nova pergunta."

    correct_option = current_question.get("correct_option")
    if selected_option == correct_option:
        return "Correto! 🎉"
    else:
        return f"Incorreto. A resposta correta era: {correct_option}."






# Criação da interface Gradio
with gr.Blocks(theme="Soft") as app:

    # Título da app
    gr.Markdown("# ToLearn")
    gr.Markdown("## Taxonomia de Bloom - Quiz Interativo")

    # Seletor de matérias
    materia_seletor = gr.Dropdown(
        label="Selecione a Matéria",
        choices=["Matemática", "História", "Biologia"],
        value="Matemática",
    )

    materia_status = gr.Textbox(label="Status Banco de Dados:", interactive=False)

    # Lembrar
    gr.Markdown('# Lembrar')

    # Espaço o Que vc lembra?
    input_lembro = gr.Textbox(label="Digite a baixo, o que você lembra sobre o conteúdo estudado.")
    botao_submeter = gr.Button("(X)")

    parabens = gr.Textbox(label="(X):", interactive=False)

    # Conteúdo de Hoje
    botao_submeter_conteudo = gr.Button("(X)")
    content_1 = gr.Textbox(label="(X):", interactive=False)

    # Comparação
    compare = gr.Button("Comparação")
    resultados = gr.Textbox(label="Comparação do conteudo_de_hoje:", interactive=False)

    # Compreender
    gr.Markdown('# Compreender')

    # Texto dizendo "Complete a frase a seguir para exercitar os conceitos da aula de hoje: "
    # Espaço para gerar a frase a ser completada
    botao_complete = gr.Button("complete a frase")
    frase = gr.Textbox(label="frase:", interactive=True)
    # Resposta
    # Resultado

    # Aplicar
    gr.Markdown('# Aplicar')

    # Botão para gerar pergunta
    botao_gerar = gr.Button("Gere uma nova pergunta")
    pergunta_formatada = gr.Textbox(label="Pergunta Gerada:", interactive=False)

    # Campo interativo para seleção de resposta com botões mais intuitivos
    opcoes_respostas = gr.Radio(label="Selecione a resposta correta:", choices=["A", "B", "C", "D"], interactive=True, type="value") #

    # Botão para verificar a resposta
    botao_verificar = gr.Button("Verificar resposta")
    feedback_resposta = gr.Textbox(label="Resultado:", interactive=False)




    # Conectando componentes às funções
    materia_seletor.change(
        fn=define_database,
        inputs=[materia_seletor],
        outputs=[materia_status]
    )
    botao_submeter.click(
        fn=oq_lembro,
        inputs=input_lembro,
        outputs=[parabens] #parabens
    )
    compare.click(
        fn=avaliar_respostas,
        inputs=[input_lembro, content_1], # Passing the Gradio components
        outputs=resultados
    )
    botao_submeter_conteudo.click(
        fn=puxar_conteudo,
        inputs=[],
        outputs=[content_1] #content_1
    )
    # Conexção entre função compreender e espaço do gradio
    botao_complete.click(
        fn=complete_frase,
        inputs=[],
        outputs=[frase] #content_1
    )
    botao_gerar.click(
        fn=gerar_pergunta,
        inputs=[],
        outputs=[pergunta_formatada, opcoes_respostas, feedback_resposta]
    )
    materia_seletor.change(
        fn=gerar_pergunta,
        inputs=[],
        outputs=[opcoes_respostas]
    )
    botao_verificar.click(
        fn=verificar_resposta,
        inputs=[opcoes_respostas],
        outputs=[feedback_resposta]
    )



# Inicializando a aplicação
if __name__ == "__main__":
    app.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d6e1cc840beb9c74a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Aplicar (OK)

In [ ]:
#Alternativa 4



# Variáveis globais
selected_subject = ""
current_question = {}
db_map = {
    "Matemática": "banco_matematica.db",
    "História": "banco_historia.db",
    "Biologia": "banco_biologia.db",
}



# Função 1 para definir banco de dados com base na matéria
def define_database(subject):
    global selected_subject
    selected_subject = subject
    return f"Banco de dados definido para {db_map.get(subject, 'desconhecido')}."

# Função 2 para gerar uma nova pergunta
def gerar_pergunta():
    global current_question

    if not selected_subject:
        return "Por favor, selecione uma matéria antes de gerar perguntas.", [""], ""

    db_path = db_map.get(selected_subject)
    if not db_path:
        return "Banco de dados não encontrado para a matéria selecionada.", [""], ""

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT question, option_a, option_b, option_c, option_d, correct_option FROM questions ORDER BY RANDOM() LIMIT 1;")
        row = cursor.fetchone()
        conn.close()

        if not row:
            return "Nenhuma pergunta disponível no banco de dados.", [""], ""

        # Guardar a pergunta atual globalmente
        current_question = {
            "question": row[0],
            "options": [row[1], row[2], row[3], row[4]],
            "correct_option": row[5],
        }
        question, option_a, option_b, option_c, option_d, correct_option = row
        pergunta_formatada = (
            f"Pergunta: {question}\n\n"
            f"A) {option_a}\n"
            f"B) {option_b}\n"
            f"C) {option_c}\n"
            f"D) {option_d}"
        )

        return pergunta_formatada, current_question['options'], "" ,

    except sqlite3.Error as e:
        return f"Erro ao acessar o banco de dados: {str(e)}", [""], ""

# Função 3 para verificar a resposta
def verificar_resposta(selected_option):
    if not current_question:
        return "Nenhuma pergunta disponível. Gere uma nova pergunta."

    correct_option = current_question.get("correct_option")
    if selected_option == correct_option:
        return "Correto! 🎉"
    else:
        return f"Incorreto. A resposta correta era: {correct_option}."






# Criação da interface Gradio
with gr.Blocks(theme="Soft") as app:

    # Título da app
    gr.Markdown("# ToLearn")
    gr.Markdown("## Taxonomia de Bloom - Quiz Interativo")

    # Seletor de matérias
    materia_seletor = gr.Dropdown(
        label="Selecione a Matéria",
        choices=["Matemática", "História", "Biologia"],
        value="Matemática",
    )
    materia_status = gr.Textbox(label="Banco de Dados Configurado:", interactive=False)

    # Botão para gerar pergunta
    botao_gerar = gr.Button("Gere uma nova pergunta")
    pergunta_formatada = gr.Textbox(label="Pergunta Gerada:", interactive=False)

    # Campo interativo para seleção de resposta com botões mais intuitivos
    opcoes_respostas = gr.Radio(label="Selecione a resposta correta:", choices=["A", "B", "C", "D"], interactive=True, type="value") #

    # Botão para verificar a resposta
    botao_verificar = gr.Button("Verificar resposta")
    feedback_resposta = gr.Textbox(label="Resultado:", interactive=False)




    # Conectando componentes às funções
    materia_seletor.change(
        fn=define_database,
        inputs=[materia_seletor],
        outputs=[materia_status]
    )
    botao_gerar.click(
        fn=gerar_pergunta,
        inputs=[],
        outputs=[pergunta_formatada, opcoes_respostas, feedback_resposta]
    )
    materia_seletor.change(
        fn=gerar_pergunta,
        inputs=[],
        outputs=[opcoes_respostas]
    )
    botao_verificar.click(
        fn=verificar_resposta,
        inputs=[opcoes_respostas],
        outputs=[feedback_resposta]
    )


# Inicializando a aplicação
if __name__ == "__main__":
    app.launch(debug=True)


### Analisar


Problemas do mundo real que aplicam o conteúdo estudado.

#### Teste prática de geração de questões por IA (2.0)

### Avaliar


In [ ]:
print('resumo_1: ')

In [ ]:
#corrija seus erros

### Criar

In [ ]:
print('A partir do conteúdo estudado nesta aula, elabore um Mapa Mental.')
#print('A partir do conteúdo estudado nesta aula, elabore um Flash Card.')
#print('A partir do conteúdo estudado nesta aula, elabore 3 rodados no pensamento difuso.')

## Exit


In [ ]:
print(f'Parabéns pelo seu avanço, veja algumas métricas sobre os seus estudos: {metricas}, continue se dedicando, bons estudos!')